In [1]:
import re
import numpy as np
from fractions import Fraction
from math import gcd
from functools import reduce
def formula_in_dict(formula:str)-> dict:
    element_pattern = r"([A-Z][a-z]*)"
    number_pattern = r"(\d*)"
    pattern = element_pattern + number_pattern
    counts = {}
    for (element, count) in re.findall(pattern, formula):
        if count == "":
            count = 1
        else:
            count = int(count)
        if element in counts:
            counts[element] += count
        else:
            counts[element] = count
    return counts


def balance_reaction(reactants: list, products: list) -> list:
    all_species = reactants + products
    composition_dicts = [formula_in_dict(f) for f in all_species]
    all_elements = set()
    for composition_dict in composition_dicts:
        for elem in composition_dict.keys():
            all_elements.add(elem)
    elements = sorted(all_elements)

    matrix = []
    for elem in elements:
        row = []
        for mol_dict in composition_dicts:
             row.append(mol_dict.get(elem, 0))
        for i in range(len(reactants), len(all_species)):
            row[i] *= -1
        matrix.append(row)

    A = np.array(matrix, dtype=float)
    u, s, vh = np.linalg.svd(A)
    vec = vh[-1]
    tol = 1e-10
    non_zero_vals = vec[np.abs(vec) > tol]
    min_val = np.min(np.abs(non_zero_vals)) 
    vec_scaled = vec / min_val
    coeffs_frac = [Fraction(c).limit_denominator() for c in vec_scaled]
    denominators = [f.denominator for f in coeffs_frac]
    def lcm(a,b): return abs(a*b)//gcd(a,b)
    lcm_total = reduce(lcm, denominators, 1)
    coeffs_int = [int(f * lcm_total) for f in coeffs_frac]

   

    non_zero_ints = [abs(x) for x in coeffs_int if x != 0]
    pgcd_total = reduce(gcd, non_zero_ints)
    coeffs_final = [x // pgcd_total for x in coeffs_int]

    if any(x < 0 for x in coeffs_final):
        coeffs_final = [-x for x in coeffs_final]


    return coeffs_final[:len(reactants)], coeffs_final[len(reactants):]

print(balance_reaction(["CaO2H2", "H2SO4"], ["CaSO4", "H2O"]))




([1, 1], [1, 2])
